# Sentiment Analysis

### Twitter Terms

April 2019

Based on original script [here](https://github.com/cipher813/bitcoin_prediction/blob/master/scripts/tweethack1.py).

In [1]:
import re
import tweepy
import pandas as pd
from textblob import TextBlob
from collections import defaultdict

In [2]:
class TwitterClient:
    def __init__(self):
        # store these in ~/.bash_profile
        self.consumer_key = %env TWITTER_API_KEY
        self.consumer_secret = %env TWITTER_API_SECRET_KEY
        self.access_token = %env TWITTER_ACCESS_TOKEN
        self.access_secret = %env TWITTER_ACCESS_TOKEN_SECRET

        self.auth = tweepy.OAuthHandler(self.consumer_key, self.consumer_secret)
        self.auth.set_access_token(self.access_token, self.access_secret)

        self.api = tweepy.API(self.auth)
        
    def get_tweets(self, term):
        """Get the latest set of tweets by term, grouped by sentiment.
        
        :term: str Term to match in tweets
        """
        td = defaultdict(list)
        tweets = self.api.search(term,tweet_mode="extended")
        for tweet in tweets:
            if 'retweeted_status' in tweet._json:
                txt = TextBlob(tweet._json['retweeted_status']['full_text'])
            else:
                txt = TextBlob(tweet.full_text)
            polarity = txt.sentiment.polarity
            if polarity > 0:
                td['pos'].append(f"{txt} ({polarity:.3f})")
            elif polarity < 0:
                td['neg'].append(f"{txt} ({polarity:.3f})")
            else:
                td['neu'].append(f"{txt}-{polarity:.3f}")
        pos, neg, neu = len(td['pos']),len(td['neg']),len(td['neu'])
        tot = pos+neg+neu
        print(f"Tweet stats:\nPositive:{pos}, {pos/tot*100:.1f}%\nNegative:{neg}, {neg/tot*100:.1f}%\nNeutral:{neu}, {neu/tot*100:.1f}%")
        return td
    
    def clean_tweet(self, tweet):
        """Format tweet into clean string.
        
        :tweet: str tweet to clean
        """
        tweet = ' '.join(re.sub(r"(.+@[A-Za-z0-9_]+)|([^0-9A-Za-z\.\s\t]) |(\w+:\/\/\S+)", "", tweet).split()) 
        return tweet

In [3]:
twitter = TwitterClient()
d = twitter.get_tweets("Trump")

Tweet stats:
Positive:5, 35.7%
Negative:5, 35.7%
Neutral:4, 28.6%


In [4]:
tweet = d['pos'][0]
tweet

'One can be conservative and not support Trump but I honestly don’t remember the last time Jennifer expressed a conservative thought. https://t.co/AyemuX3YAu (0.30)'

In [5]:
tweet = twitter.clean_tweet(d['pos'][0])
tweet

'One can be conservative and not support Trump but I honestly don’t remember the last time Jennifer expressed a conservative thought. (0.30)'

In [6]:
d['pos']

['One can be conservative and not support Trump but I honestly don’t remember the last time Jennifer expressed a conservative thought. https://t.co/AyemuX3YAu (0.30)',
 '@ananavarro @markmobility @realDonaldTrump @TifaniRoberts Do you EVER get tired of lying? Trump CONDEMNS MS-13 for this kind of thuggery. He is working to stop it from happening by securing the border; propagandists like yourself are more concerned with making Trump look bad than with American safety. (0.00)',
 "When a President lacks character, strong oversight is essential.\n\nHouse Republicans seem to think as long as you can't prove a crime, all is fair in love and war, and what the Trump campaign did is all okay.\n\nI don't feel that way. And I don't think most Americans feel that way. https://t.co/KZpLkIvshx (0.37)",
 'Do you have any evidence Obama ordered the FBI to surveil Trump? Please stop making stuff up as distractions, @mschlapp. Even if it’s good for business. Have a nice Sunday. https://t.co/aQTEmZ0tCL 

In [7]:
d['neg']

['Looks like Bob Mueller’s team of 13 Trump Haters &amp; Angry Democrats are illegally leaking information to the press while the Fake News Media make up their own stories with or without sources - sources no longer matter to our corrupt &amp; dishonest Mainstream Media, they are a Joke! (-0.30)',
 "@FoxNewsSunday Trump said he will not that he could. Then it was oh I cant drop it I'm being audited. Now its I won so no need to drop them. If you ain't got shit to hide then drop them and throw it in the dems face but we all know why he wont (-0.20)",
 "Look at this stupid A$$ white liberal calling a Mexican woman a racist and harrasing her for wearing a Trump hat. \n\nWhite liberals think they control minorities, and it's always them harrasing minorities who don't cowtow to their regressive bullsh*t! \n\n https://t.co/zV71JDnR6R (-0.20)",
 "Look at this stupid A$$ white liberal calling a Mexican woman a racist and harrasing her for wearing a Trump hat. \n\nWhite liberals think they contr

In [8]:
d['neu']

['President. Donald. Trump. https://t.co/jcQnQTMTYv-0.00',
 '@CasanovaInk @DevlinBarrett @gtconway3d I asked a follower if he caught trump growing his grandmother what would he do? He says nothing, he may be trying to heal her cance r-0.00',
 "Dictators fear what they can't buy, bully, or control. It's why Putin loves Trump. https://t.co/Dk4frRqyl5-0.00",
 'Raise your hand if you\'ve NEVER said the words "President Trump" 👋-0.00']